# Procesador de archivos

In [34]:
import utils_rag as ur
import os
import json
from concurrent.futures import ThreadPoolExecutor

In [21]:

def get_documents_by_extensions(folder_path, extensions):
    """
    Obtiene los nombres de los documentos en una carpeta que terminan con una o varias extensiones específicas.

    :param folder_path: Ruta de la carpeta
    :param extensions: Lista de extensiones de archivo (por ejemplo, ['.txt', '.pdf'])
    :return: Lista de nombres de archivos con las extensiones especificadas
    """
    documents = []
    for filename in os.listdir(folder_path):
        if any(filename.endswith(ext) for ext in extensions):
            documents.append(folder_path+'/'+filename)
    return documents

# Ejemplo de uso
folder_path = 'data/docs'
extensions = ['.txt', '.pdf']
print(get_documents_by_extensions(folder_path, extensions))
file_paths=get_documents_by_extensions(folder_path, extensions)

['data/docs/DT_Golondrinas.pdf', 'data/docs/Implementacion-y-aplicaciones-de-un-sistema-de-codificacion-automatica-de-la-lista-de-espera-chilena.pdf', 'data/docs/DeepSeek_R1.pdf']


In [36]:
def generate_file_paths_and_dicts(file_paths):
    """
    Genera las rutas de salida y un diccionario con la estructura especificada para una lista de archivos.

    :param file_paths: Lista de rutas de archivos de entrada
    :return: Diccionario con las rutas generadas para cada archivo
    """
    dict_files = {}
    
    for i, file_path in enumerate(file_paths, start=1):
        base_name = os.path.basename(file_path)
        name, ext = os.path.splitext(base_name)
        
     
        output_file_2 = os.path.join("data/chunks_docs", f"chunks_{name}.json")
        
        dict_files[f"file{i}"] = {
            "file_path": file_path,
            "output_path": output_file_2,
        }

    with open("data/chunks_docs/dict_pdfs.json", 'w') as json_file:
        json.dump(dict_files, json_file, indent=4)
    
    return dict_files

In [37]:
dict_pdfs=generate_file_paths_and_dicts(file_paths)
print(dict_pdfs)

{'file1': {'file_path': 'data/docs/DT_Golondrinas.pdf', 'output_path': 'data/chunks_docs/chunks_DT_Golondrinas.json'}, 'file2': {'file_path': 'data/docs/Implementacion-y-aplicaciones-de-un-sistema-de-codificacion-automatica-de-la-lista-de-espera-chilena.pdf', 'output_path': 'data/chunks_docs/chunks_Implementacion-y-aplicaciones-de-un-sistema-de-codificacion-automatica-de-la-lista-de-espera-chilena.json'}, 'file3': {'file_path': 'data/docs/DeepSeek_R1.pdf', 'output_path': 'data/chunks_docs/chunks_DeepSeek_R1.json'}}


In [32]:


def process_document(file_info, ejecutar_pdf):
    """
    Procesa un documento si el archivo de salida no existe.

    :param file_info: Diccionario con la información del archivo
    :param ejecutar_pdf: Parámetro para la función ejecutar_chunking_pdf
    :return: True si el archivo fue procesado, False si ya existía el archivo de salida
    """
    output_path = file_info['output_path']
    if os.path.isfile(output_path):
        return False
    else:
        uu.ejecutar_chunking_pdf(file_info, ejecutar_pdf=ejecutar_pdf)
        return True

def process_documents(dict_pdfs, ejecutar_pdf, parallel=False):
    """
    Procesa una lista de documentos, verificando si el archivo de salida existe.

    :param dict_pdfs: Diccionario con la información de los archivos
    :param ejecutar_pdf: Parámetro para la función ejecutar_chunking_pdf
    :param parallel: Booleano para indicar si se debe paralelizar el procesamiento
    :return: Lista de resultados indicando si cada archivo fue procesado
    """
    if parallel:
        with ThreadPoolExecutor() as executor:
            results = list(executor.map(lambda item: process_document(item[1], ejecutar_pdf), dict_pdfs.items()))
    else:
        results = [process_document(file_info, ejecutar_pdf) for file_info in dict_pdfs.values()]
    
    return results

In [33]:
ejecutar_pdf = True
print(process_documents(dict_pdfs, ejecutar_pdf, parallel=True))

[True, True, False]
